In [14]:
import time
import numpy as np

In [134]:
# レベル1
problem = '''
-35-9--48
--9--8--3
-4-6-5--1
----74---
-2-----6-
---15----
8--9-2-7-
9--5--2--
61--4-53-
'''

In [29]:
# レベル2
# https://si-coding.net/sudoku10.html
problem = '''
------1-4
4---78---
-3------2
5--4-7-3-
2------5-
9---264--
-27------
--56----7
-6--13---
'''

In [146]:
# レベル3
# https://www.j-cast.com/premium/2018/10/19341452.html?p=all
problem = '''
---------
3--2-9--7
94--1--82
-5-6-2-3-
--63-74--
---------
---------
-82-3-59-
1--9-4--6
'''

In [127]:
# 世界一難しい？
# https://www.conceptispuzzles.com/ja/index.aspx?uri=info/article/424
problem = '''
8--------
--36-----
-7--9-2--
-5---7---
----457--
---1---3-
--1----68
--85---1-
-9----4--
'''

In [125]:
# レベル?
# https://puzzlemadness.co.uk/16by16giantsudoku/
problem = '''
8---F2G----C----
---CE--8----6-4-
-7F59----1------
3-B-------5-2--E
D-6A--8G---7---5
E---D-A------6G9
-83---------D-B-
G-9-7342--8D---C
932-----B---FGC-
6----1--58---3--
C----4-A--6E--1-
B-5D---F--73-4--
7C--2-----E---63
-----56---98B7--
---3AG9--F---D-1
----1F3--7---A--
'''

In [147]:
numbers = {'-': 0}
numbers.update({ str(i):i for i in range(1, 10)})
numbers.update({ s: 10 + ord(s) - ord('A') for s in 'ABCDEFG' })
problem = [numbers[c] for line in problem.split('\n') if line != '' for c in line]
problem = np.asarray(problem, dtype='int32')
size = int(np.sqrt(problem.size))
problem = problem.reshape((size, size))
print(problem)

[[0 0 0 0 0 0 0 0 0]
 [3 0 0 2 0 9 0 0 7]
 [9 4 0 0 1 0 0 8 2]
 [0 5 0 6 0 2 0 3 0]
 [0 0 6 3 0 7 4 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 8 2 0 3 0 5 9 0]
 [1 0 0 9 0 4 0 0 6]]


In [148]:
def sudoku_solve(problem):
    rows, cols = problem.shape
    all_nums = set([i for i in range(0, 10)])
    for i in range(rows):
        for j in range(cols):
            if problem[i, j] == 0:
                k = i // 3
                l = j // 3
                row_nums = problem[i, :].tolist()
                col_nums = problem[:, j].tolist()
                blk_nums = problem[3*k:3*k+3, 3*l:3*l+3].reshape(-1).tolist()
                used_nums = set(row_nums + col_nums + blk_nums)
                nums = all_nums.difference(used_nums)
                if len(nums) == 0:
                    return False
                
                success = False
                for n in nums:
                    problem[i, j] = n
                    if sudoku_solve(problem):
                        success = True
                        break

                    problem[i, j] = 0

                if not success:
                    return False
        
    return True

In [149]:
def sudoku_solve2(problem, start = 0):
    rows, cols = problem.shape
    if rows != cols:
        raise Exception('Given problem is not square!')
        
    size = rows
    sub = int(np.sqrt(size))
    nums = set([i for i in range(0, size + 1)])
    for c in range(start, size * size):
        i = c // size
        j = c % size
        if problem[i, j] == 0:
            k = i // sub
            l = j // sub
            row_nums = problem[i, :].tolist()
            col_nums = problem[:, j].tolist()
            blk_nums = problem[sub*k:sub*k+sub, sub*l:sub*l+sub].reshape(-1).tolist()
            used = set(row_nums + col_nums + blk_nums)
            diff = nums.difference(used)
            if len(diff) == 0:
                return False

            success = False
            for n in diff:
                problem[i, j] = n
                if sudoku_solve2(problem, i * cols + j + 1):
                    success = True
                    break

                problem[i, j] = 0

            if not success:
                return False
        
    return True

In [153]:
# 時間計測開始
start_time = time.perf_counter()
# 解く
answer = problem.copy()
success = sudoku_solve2(answer)
# 時間計測終了
end_time = time.perf_counter()
elapsed_ms = 1000.0 * (end_time - start_time)
# 結果を表示
print('{:.8f} msec'.format(elapsed_ms))
if success:
    print(answer)
else:
    print('Cannot solve this sudoku!')

236.05478300 msec
[[5 2 1 7 6 8 3 4 9]
 [3 6 8 2 4 9 1 5 7]
 [9 4 7 5 1 3 6 8 2]
 [4 5 9 6 8 2 7 3 1]
 [8 1 6 3 9 7 4 2 5]
 [2 7 3 4 5 1 9 6 8]
 [6 9 4 8 7 5 2 1 3]
 [7 8 2 1 3 6 5 9 4]
 [1 3 5 9 2 4 8 7 6]]
